In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import os
from keras.applications.vgg16 import VGG16
import seaborn as sns


In [ ]:
# Reading input images and assigning labels based on folder names
print(os.listdir("data/Plants_2"))

SIZE = (200,300)  #Resize images

#Capturing training data and labels into respective lists
train_images = []
train_labels = [] 
dir = 'data/resized/train'


for label in os.listdir(dir):
    label_path = os.path.join(dir, label)
    print(label)
    for img_path in glob.glob(os.path.join(label_path, "*.JPG")):
        print('inside')    
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)


In [ ]:
print(len(train_images))
print(len(train_labels))

In [ ]:
#Converting lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)


In [ ]:
import os
import cv2

test_images = []
test_labels = [] 
test_dir = 'data/resized/valid'


for label in os.listdir(test_dir):
    if label == ".DS_Store":
        continue
    label_path = os.path.join(test_dir, label)
    print(label)
    for img_file in os.listdir(label_path):
        if img_file.endswith(('.JPG', '.jpeg')):
            img_path = os.path.join(label_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            test_images.append(img)
            test_labels.append(label)
            print(test_labels)

if len(test_images) == 0:
    print("No test images found in directory:", test_dir)
else:
    print("Test images and labels loaded successfully.")





In [ ]:
print(len(test_images))
print(len(test_labels))

In [ ]:
#Converting lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)


In [ ]:


#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:

#Loading model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0



In [ ]:

#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_training = features #This is our X input to RF


In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_for_training, y_train) #For sklearn no one hot encoding

#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)


In [ ]:

#Now predict using the trained RF model. 
prediction = model.predict(X_test_features)
#Inverse le transform to get original label back. 
prediction = le.inverse_transform(prediction)

In [ ]:

#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction))

#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels, prediction)
#print(cm)
sns.heatmap(cm, annot=True)

#Check results on a few select images
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = model.predict(input_img_features)[0] 
prediction = le.inverse_transform([prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])

In [ ]:
from sklearn.metrics import classification_report

# ...

# Now predict using the trained RF model
prediction = model.predict(X_test_features)

# Inverse le transform to get the original label back
prediction = le.inverse_transform(prediction)

# Print overall accuracy
accuracy = metrics.accuracy_score(test_labels, prediction)
print("Accuracy =", accuracy)

# Generate the classification report
report = classification_report(test_labels, prediction)

# Print the classification report
print(report)


In [ ]:
from sklearn.metrics import accuracy_score

# Compute accuracy for each class
accuracy = accuracy_score(test_labels, prediction)

# Print class-wise accuracy
print("Class-wise Accuracy:")
for i, label in enumerate(le.classes_):
    class_accuracy = accuracy_score(test_labels[test_labels == label], prediction[test_labels == label])
    print(f"{label}: {class_accuracy}")


In [ ]:
import matplotlib.pyplot as plt

# Compute class-wise accuracy
class_accuracy = []
for label in le.classes_:
    acc = accuracy_score(test_labels[test_labels == label], prediction[test_labels == label])
    class_accuracy.append(acc)

# Plot the bar plot
plt.figure(figsize=(10, 6))
plt.bar(le.classes_, class_accuracy)
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.title('Class-wise Accuracy')
plt.xticks(rotation=90)
plt.show()



In [ ]:
from sklearn.metrics import roc_curve, auc

# Compute the probabilities for each class
prob = model.predict_proba(X_test_features)

# Compute the ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i, label in enumerate(le.classes_):
    fpr[label], tpr[label], _ = roc_curve(test_labels_encoded, prob[:, i], pos_label=i)
    roc_auc[label] = auc(fpr[label], tpr[label])

# Plot the ROC curves
plt.figure(figsize=(10, 6))
for label in le.classes_:
    plt.plot(fpr[label], tpr[label], label=f"{label} (AUC = {roc_auc[label]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()
